# Text pre-processing

Text can come in a variety of forms from a list of individual words, to sentences to multiple paragraphs with special characters (like tweets for example). This step is very important in pipeline which helps to feed right data to the model.
Text preprocessing seems severely overlooked topic. A few people I spoke to mentioned inconsistent results from their NLP applications only to realize that they were not  preprocessing their text or were using the wrong kind of text preprocessing for their project.

## Table of content:
-  Introduction
-  Dataset
-  Types of text preprocessing techniques
   - Lowercasing
   - Normalization

## Introduction

In this tutorial, you will discover how you can clean and prepare your text ready for modeling with machine learning.

There is a whole suite of text preparation methods that you may need to use, and the choice of methods really depends on your natural language processing task.

## Types of text preprocessing techniques


## Normalization

Text normalization is the process of transforming text into a canonical (standard) form. For example, the word “loooove” and “luv” can be transformed to “love”, its canonical form. Another example is mapping of near identical words such as “stopwords”, “stop-words” and “stop words” to just “stopwords”.

Text normalization is important for noisy texts such as social media comments, text messages and comments to blog posts where abbreviations, misspellings and use of out-of-vocabulary words (oov) are prevalent.

Unicode normalization is one of the techinque where special character has to be normalized.